# Sentiment analysis

##  Set up

In [1]:
%pip install numpy pandas nltk sklearn scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

## Load dataset

In [3]:
df = pd.read_csv(r'../resources/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

## Tiền xử lý dữ liệu

### Làm sạch dữ liệu 
- Loại bỏ các thẻ HTML
- Loại bỏ khoảng trắng thừa và dấu câu 
- Loại bỏ stop words
- Chuyển đổi chữ hoa thành chữ thường 
- Chuyển đổi nhãn sang số

#### Loại bỏ thẻ HTML

In [5]:
df['review'] = df['review'].str.replace("<br />", "")
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

#### Loại bỏ khoảng trắng thừa và dấu câu

In [6]:
df['review'] = df['review'].str.translate(str.maketrans('', '', string.punctuation))

In [7]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

In [8]:
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.replace(r'\s+',' ', regex=True)
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

#### Loại bỏ stop words

In [9]:
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [11]:
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)

In [12]:
df['review'] = df['review'].apply(remove_stop_words)
df['review'][2]

'I thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy The plot simplistic dialogue witty characters likable even well bread suspected serial killer While may disappointed realize Match Point 2 Risk Addiction I thought proof Woody Allen still fully control style many us grown loveThis Id laughed one Woodys comedies years dare I say decade While Ive never impressed Scarlet Johanson managed tone sexy image jumped right average spirited young womanThis may crown jewel career wittier Devil Wears Prada interesting Superman great comedy go see friends'

#### Chuyển chữ hoa thành chữ thường 

In [13]:
df['review'] = df['review'].str.lower()
df['review'][2]

'i thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy the plot simplistic dialogue witty characters likable even well bread suspected serial killer while may disappointed realize match point 2 risk addiction i thought proof woody allen still fully control style many us grown lovethis id laughed one woodys comedies years dare i say decade while ive never impressed scarlet johanson managed tone sexy image jumped right average spirited young womanthis may crown jewel career wittier devil wears prada interesting superman great comedy go see friends'

#### Chuyển đổi nhãn sang số

In [14]:
def sentiment_encode(text):
    if(text == "positive"):
        return 1
    return -1

In [15]:
df['sentiment'] = df['sentiment'].apply(sentiment_encode)

### Chia tập dữ liệu

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [17]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=42)

In [18]:
# Chuyển đổi văn bản thành các đặc trưng số sử dụng TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [19]:
X_train_tfidf[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 47 stored elements and shape (1, 5000)>

In [20]:
sentiment = y_train.unique()
sentiment

array([-1,  1])

## Mô hình

### Decision tree (Không tham số)

In [21]:
# Huấn luyện mô hình Decision Tree
clf = DecisionTreeClassifier(
    random_state=42, 
)
clf.fit(X_train_tfidf, y_train)

DecisionTreeClassifier(random_state=42)

In [22]:
from sklearn.metrics import accuracy_score, classification_report

# Dự đoán trên tập huấn luyện
y_pred_train = clf.predict(X_train_tfidf)

# Tính toán độ chính xác và báo cáo phân loại cho tập huấn luyện
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

# In kết quả cho tập huấn luyện
print(f'Accuracy on training set: {accuracy_train}')
print('Classification Report on training set:')
print(report_train)

# Dự đoán trên tập kiểm tra
y_pred_test = clf.predict(X_test_tfidf)

# Tính toán độ chính xác và báo cáo phân loại cho tập kiểm tra
accuracy_test = accuracy_score(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test)

# In kết quả cho tập kiểm tra
print(f'Accuracy on test set: {accuracy_test}')
print('Classification Report on test set:')
print(report_test)

Accuracy on training set: 1.0
Classification Report on training set:
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     17589
           1       1.00      1.00      1.00     17411

    accuracy                           1.00     35000
   macro avg       1.00      1.00      1.00     35000
weighted avg       1.00      1.00      1.00     35000

Accuracy on test set: 0.7228
Classification Report on test set:
              precision    recall  f1-score   support

          -1       0.72      0.73      0.72      7411
           1       0.73      0.72      0.72      7589

    accuracy                           0.72     15000
   macro avg       0.72      0.72      0.72     15000
weighted avg       0.72      0.72      0.72     15000



In [23]:
print(f'Depth of the tree: {clf.get_depth()}')
print(f'Number of leaves: {clf.get_n_leaves()}')
print(f'Number of features: {clf.n_features_in_}')
print(f'Feature importances: {clf.feature_importances_}')
print(f'max_depth: {clf.max_depth}')
print(f'min_samples_split: {clf.min_samples_split}')
print(f'min_samples_leaf: {clf.min_samples_leaf}')
print(f'criterion: {clf.criterion}')

Depth of the tree: 177
Number of leaves: 3346
Number of features: 5000
Feature importances: [0.00000000e+00 7.61924469e-05 0.00000000e+00 ... 2.16488730e-04
 0.00000000e+00 0.00000000e+00]
max_depth: None
min_samples_split: 2
min_samples_leaf: 1
criterion: gini


### Áp dụng phương pháp GridSearch chọn các hyperparameter

In [24]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'max_depth': [20, 30, 40],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3],
    'criterion': ['gini', 'entropy'],
}

grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5),
)
grid_search.fit(X_train_tfidf, y_train)
print(grid_search.best_params_)


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5}


In [25]:
print(grid_search.best_estimator_)

DecisionTreeClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=5)


#### Độ chính xác của mô hình với cách chọn tham số Grid Search

In [26]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small training set using the best estimator
y_pred_train = grid_search.best_estimator_.predict(X_train_tfidf)

# Calculate accuracy and classification report
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.8352571428571428
Classification Report:
              precision    recall  f1-score   support

          -1       0.90      0.75      0.82     17589
           1       0.79      0.92      0.85     17411

    accuracy                           0.84     35000
   macro avg       0.84      0.84      0.83     35000
weighted avg       0.84      0.84      0.83     35000



In [27]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small test set using the best estimator
y_pred = grid_search.best_estimator_.predict(X_test_tfidf)

# Calculate accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.7463333333333333
Classification Report:
              precision    recall  f1-score   support

          -1       0.79      0.66      0.72      7411
           1       0.72      0.83      0.77      7589

    accuracy                           0.75     15000
   macro avg       0.75      0.75      0.74     15000
weighted avg       0.75      0.75      0.74     15000



#### Kiểm tra các thuộc tính của cây 

In [28]:
best_clf = grid_search.best_estimator_

print(f'Depth of the tree: {best_clf.get_depth()}')
print(f'Number of leaves: {best_clf.get_n_leaves()}')
print(f'Number of features: {best_clf.n_features_in_}')
print(f'Feature importances: {best_clf.feature_importances_}')

Depth of the tree: 20
Number of leaves: 879
Number of features: 5000
Feature importances: [0. 0. 0. ... 0. 0. 0.]


### Bonus: Train trên dữ liệu nhỏ

In [29]:
import pandas as pd
from scipy.sparse import csr_matrix

# Convert sparse matrices to dense DataFrames
X_train_tfidf_df = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf)
X_test_tfidf_df = pd.DataFrame.sparse.from_spmatrix(X_test_tfidf)

# Reset indices to ensure alignment
X_train_tfidf_df.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test_tfidf_df.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Sample from the dense DataFrames
small_X_train_df = X_train_tfidf_df.sample(n=5000, random_state=42)
small_y_train = y_train.loc[small_X_train_df.index]

small_X_test_df = X_test_tfidf_df.sample(n=1500 , random_state=42)
small_y_test = y_test.loc[small_X_test_df.index]

# Convert back to sparse matrices if needed
small_X_train = csr_matrix(small_X_train_df)
small_X_test = csr_matrix(small_X_test_df)

In [31]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'max_depth': [20, 30, 40],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2, 3],
    'criterion': ['gini', 'entropy'],
}

grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5),
)
grid_search.fit(small_X_train, small_y_train)
print(grid_search.best_params_)

{'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 3}


In [32]:
print(grid_search.best_estimator_)

DecisionTreeClassifier(criterion='entropy', max_depth=20, min_samples_leaf=2,
                       min_samples_split=3)


#### Độ chính xác của cách chọn tham số Grid Search trên tập dữ liệu nhỏ

In [33]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small training set using the best estimator
small_y_pred_train = grid_search.best_estimator_.predict(small_X_train)

# Calculate accuracy and classification report
accuracy_train = accuracy_score(small_y_train, small_y_pred_train)
report_train = classification_report(small_y_train, small_y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.887
Classification Report:
              precision    recall  f1-score   support

          -1       0.96      0.81      0.88      2525
           1       0.83      0.96      0.89      2475

    accuracy                           0.89      5000
   macro avg       0.90      0.89      0.89      5000
weighted avg       0.90      0.89      0.89      5000



In [34]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small test set using the best estimator
small_y_pred = grid_search.best_estimator_.predict(small_X_test)

# Calculate accuracy and classification report
accuracy = accuracy_score(small_y_test, small_y_pred)
report = classification_report(small_y_test, small_y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.6826666666666666
Classification Report:
              precision    recall  f1-score   support

          -1       0.72      0.59      0.65       740
           1       0.66      0.77      0.71       760

    accuracy                           0.68      1500
   macro avg       0.69      0.68      0.68      1500
weighted avg       0.69      0.68      0.68      1500

